# Install Dependecies

#### Lime

In [3]:
!pip install lime

     |████████████████████████████████| 275 kB 5.9 MB/s 
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283857 sha256=c21660e20267dcb82cca08df7a4866042447fa1610a352538856d5d142429daf
  Stored in directory: /root/.cache/pip/wheels/ca/cb/e5/ac701e12d365a08917bf4c6171c0961bc880a8181359c66aa7
Successfully built lime


In [4]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 41.2 MB/s 
     |████████████████████████████████| 56 kB 4.4 MB/s 
     |████████████████████████████████| 895 kB 32.1 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
!pip install stanza

     |████████████████████████████████| 432 kB 5.4 MB/s 
     |████████████████████████████████| 170 kB 37.9 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=69d20f83d3bf2b9c25a1c3538017c8c2a09e82baf9399629329414c1039490d2
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [6]:
!pip install spacy
!spacy download en

     |████████████████████████████████| 12.0 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


#### Langid

In [7]:
!pip install langid

     |████████████████████████████████| 1.9 MB 1.2 MB/s 
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941187 sha256=6bedec4d26effe6a37ab98d3b1f3739eb4418378c8f8febd539a5238e54c2742
  Stored in directory: /root/.cache/pip/wheels/2b/bb/7f/11e4db39477278161e882eadc46fb558949a28b13470fc74b8
Successfully built langid


# Import Libraries

In [8]:
import spacy
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
import transformers

import stanza
import nltk
from sklearn import feature_selection
from sklearn import metrics


# Download Data

In [9]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('rslp')


stanza.download('en')
nlp = stanza.Pipeline('en')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


2021-10-29 11:12:50 INFO: Downloading default packages for language: en (English)...


2021-10-29 11:13:01 INFO: Finished downloading models and saved to /root/stanza_resources.
2021-10-29 11:13:01 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| depparse     | combined  |
| sentiment    | sstplus   |
| constituency | wsj       |
| ner          | ontonotes |

2021-10-29 11:13:01 INFO: Use device: gpu
2021-10-29 11:13:01 INFO: Loading: tokenize
2021-10-29 11:13:15 INFO: Loading: pos
2021-10-29 11:13:16 INFO: Loading: lemma
2021-10-29 11:13:16 INFO: Loading: depparse
2021-10-29 11:13:16 INFO: Loading: sentiment
2021-10-29 11:13:17 INFO: Loading: constituency
2021-10-29 11:13:18 INFO: Loading: ner
2021-10-29 11:13:18 INFO: Done loading processors!


In [11]:
import random

p = 0.01

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [12]:
%%time
dtf = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset/tweets.csv", skiprows=lambda i: i>0 and random.random() > p, error_bad_lines=False, nrows=250000, sep=';', lineterminator='\n')



<string>:2: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 20 s, sys: 2.77 s, total: 22.7 s
Wall time: 1min 12s


In [69]:
dtf.head()

,id,user,fullname,url,timestamp,replies,likes,retweets,text\r,language,text_clean
1,1.127046e+18,mironyan,Yavruyan E,NaN,2019-05-11 01:00:33+00,0,0,0,!!!\r,en,
2,1.126824e+18,cryptobhoy,Kevin Gladstone,NaN,2019-05-10 10:19:35+00,0,0,0,@Beautyon_ Where's bitcoin sign guy when you n...,en,beautyon_ wh bitcoin sign guy need
3,1.132979e+18,MobileStarts,Mo.bi,NaN,2019-05-27 11:56:05+00,0,0,0,"New post: Top 3 Price Prediction Bitcoin, Ripp...",en,new post top 3 pric prediction bitcoin rippl e...
5,1.132980e+18,RhinoAK,RhinoAK,NaN,2019-05-27 12:00:36+00,0,0,0,Join me and start earning bitcoin by answering...,en,join start earning bitcoin answering paid emai...
7,1.132980e+18,DeribitRekt,DeribitRekt,NaN,2019-05-27 12:02:00+00,0,0,0,⏰ LIQUIDATION on BTC-PERPETUAL ☠️️\n\n Bought ...,en,liquidation btcperpet bought 10 btc 877800 27 ...


# Preprocess Text

In [36]:
%%time
spacy_nlp = spacy.load('en')

CPU times: user 593 ms, sys: 68.6 ms, total: 662 ms
Wall time: 663 ms


In [22]:
%matplotlib inline

def lemmatize_spacy(text):
  doc = spacy_nlp(text)
  lemmatize_words = []
  lemmatize_words += [token.lemma_ for token in doc]
  return lemmatize_words

####Stopwords

In [26]:
%%time
lst_stopwords = nltk.corpus.stopwords.words("english")

CPU times: user 2.37 ms, sys: 1.25 ms, total: 3.62 ms
Wall time: 3.87 ms


In [20]:
'''
Preprocess a string.
:parameter
    :param text: string - name of column containing text
    :param lst_stopwords: list - list of stopwords to remove
    :param flg_stemm: bool - whether stemming is to be applied
    :param flg_lemm: bool - whether lemmitisation is to be applied
:return
    cleaned text
'''
def lemmatize_spacy(text):
  doc = spacy_nlp(text)
  lemmatize_words = []
  lemmatize_words += [token.lemma_ for token in doc]
  return lemmatize_words

def remove_stopwords(lst_text):
  if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
  return lst_text

def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and   characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.RSLPStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
      #print(lst_text)
      lst_text =  lemmatize_spacy(" ".join(lst_text))
      """lemma_words = []
      for sent in nlp(" ".join(lst_text)).sentences:
        for word in sent.words:
          lemma_words.append(word.lemma)
        #lst_text =  [ word.lemma  for word in sent.words ]
      lst_text = lemma_words"""
      """lem = nltk.stem.wordnet.WordNetLemmatizer()
      lst_text = [lem.lemmatize(word) for word in lst_text]"""
          
    ## back to string from list
    text = " ".join(lst_text)
    return text

### Checking Tweets Language

In [ ]:
dtf.columns

In [15]:
%%time
#vê um melhor
import langid
#langid.classify("Soy muy")
dtf['language'] = dtf['text\r'].apply(lambda x: langid.classify(x)[0])


CPU times: user 4min 58s, sys: 3min 38s, total: 8min 37s
Wall time: 4min 24s


## Filter English Language

In [16]:
#filtrar para EN
dtf = dtf[dtf['language']=='en']
dtf

,id,user,fullname,url,timestamp,replies,likes,retweets,text\r,language
1,1.127046e+18,mironyan,Yavruyan E,NaN,2019-05-11 01:00:33+00,0,0,0,!!!\r,en
2,1.126824e+18,cryptobhoy,Kevin Gladstone,NaN,2019-05-10 10:19:35+00,0,0,0,@Beautyon_ Where's bitcoin sign guy when you n...,en
3,1.132979e+18,MobileStarts,Mo.bi,NaN,2019-05-27 11:56:05+00,0,0,0,"New post: Top 3 Price Prediction Bitcoin, Ripp...",en
5,1.132980e+18,RhinoAK,RhinoAK,NaN,2019-05-27 12:00:36+00,0,0,0,Join me and start earning bitcoin by answering...,en
7,1.132980e+18,DeribitRekt,DeribitRekt,NaN,2019-05-27 12:02:00+00,0,0,0,⏰ LIQUIDATION on BTC-PERPETUAL ☠️️\n\n Bought ...,en
...,...,...,...,...,...,...,...,...,...,...
169223,1.198263e+18,btc_update,Bitcoin update,NaN,2019-11-23 15:32:37+00,0,0,0,Finland Prepares For Bitcoin Ransomware Attack...,en
169224,1.198264e+18,Cryptoevolucion,Cryptovolution,NaN,2019-11-23 15:36:23+00,0,0,0,#IoTex 🚨#UPDATE🚨 . Iotex is Very Stable and lo...,en
169225,1.198265e+18,phuongdi2891,phuongdi2891,NaN,2019-11-23 15:39:21+00,0,0,0,Lydian Lion with the platform empowers human i...,en
169226,1.198265e+18,emergeen,Emergeen DN,NaN,2019-11-23 15:40:50+00,0,0,0,Bitcoin wallet\n\n1KSrHvFavm1WB682ZjJNK355nJQe...,en


In [17]:
dtf.to_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/language_en_tweets_dataset.csv')

### Stopwords Lemmatisation

In [51]:
%%time
dtf["text_clean"] = dtf["text\r"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
samples = dtf.sample(1)

CPU times: user 20min 7s, sys: 6.21 s, total: 20min 13s
Wall time: 20min 7s


#### Check Samples

In [53]:
print(remove_stopwords(samples["text\r"]))
print(list(samples["text_clean"]))

["I've been patiently waiting on Bitcoin to resurge but I'm starting to accept the harsh reality that they probably fooled everyone and were never actually here for the long game\r"]
['-PRON- have patiently wait bitcoin resurge -PRON- be start accept harsh reality probably fool everyone never actually long game']


#### Export to csv

In [54]:
dtf.to_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/stopwords_lemmatisation_tweets_preprocessing_dataset.csv')

### Stopwords Steeming

In [55]:
%%time
dtf["text_clean"] = dtf["text\r"].apply(lambda x: utils_preprocess_text(x, flg_stemm=True, flg_lemm=False, lst_stopwords=lst_stopwords))


CPU times: user 3min 27s, sys: 819 ms, total: 3min 28s
Wall time: 3min 28s


#### Check Samples

In [63]:
samples = dtf.sample(199)
print(remove_stopwords(samples["text\r"]))
print(list(samples["text_clean"]))

['Cutter Operator - Dual Graphics, Inc ( Brea, CA, USA )  - [ ➡ https://t.co/pgU2y370JC ] #Agile #Scrum #Agile #Kanban #ProjectMangement #jobs #Hiring #Careers  #Cryptocurrency #Blockchain #BTC #BitCoin #ETH #crypto https://t.co/N8LA5nE2fR\r', '$BTCUSD Analysis:\n\nImpending Bounce From Resistance Line Targets $7,800 – Bybit Margin Trading\n\nRead Here - https://t.co/1BDsx7vhAX\n\n#Bitcoin $BTC #Bybit https://t.co/AFbUX4NCW1\r', 'I wonder if the recent fall in the price of $BTC has anywhing to do with the news on Google achieving #Quantum_Supremacy.\r', 'You Can Buy a Dream Car With Bitcoin Cash at https://t.co/i3pmTYQsCD https://t.co/hOpuzpOexD 📈 Crypto Cashflow via → https://t.co/2p1Vt5799c\r', 'The latest The Online casino Daily! https://t.co/zlIGqrAKpo Thanks to @CasinosandGames @casinolifemag #bitcoin #crypto\r', '@AliceSmith725 @EXILE_AKIRA_LDH\r', 'Ask and God gives. 0,7% daily of ur invest though an bot with AI in TELEGRAM \nFrom 5 €/$ ONLY IN BTC +info:  https://t.co/CIl6GfrsX

#### Export to csv

In [64]:
dtf.to_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/stopwords_stemming_tweets_preprocessing_dataset.csv')

### Pre Processing, Stemming Lemm and Stopwords

In [46]:
%%time
dtf["text_clean"] = dtf["text\r"].apply(lambda x: utils_preprocess_text(x, flg_stemm=True, flg_lemm=True, lst_stopwords=lst_stopwords))
samples = dtf.sample(1)

CPU times: user 23min 29s, sys: 7.27 s, total: 23min 37s
Wall time: 23min 29s


In [67]:
dtf.tail()

,id,user,fullname,url,timestamp,replies,likes,retweets,text\r,language,text_clean
169223,1.198263e+18,btc_update,Bitcoin update,NaN,2019-11-23 15:32:37+00,0,0,0,Finland Prepares For Bitcoin Ransomware Attack...,en,finland prep bitcoin ransomw attack cyb fir dr...
169224,1.198264e+18,Cryptoevolucion,Cryptovolution,NaN,2019-11-23 15:36:23+00,0,0,0,#IoTex 🚨#UPDATE🚨 . Iotex is Very Stable and lo...,en,iotex updat iotex stabl look lik going uptrend...
169225,1.198265e+18,phuongdi2891,phuongdi2891,NaN,2019-11-23 15:39:21+00,0,0,0,Lydian Lion with the platform empowers human i...,en,lydian lion platform empow human interactiom r...
169226,1.198265e+18,emergeen,Emergeen DN,NaN,2019-11-23 15:40:50+00,0,0,0,Bitcoin wallet\n\n1KSrHvFavm1WB682ZjJNK355nJQe...,en,bitcoin wallet 1ksrhvfavm1wb682zjjnk355njqe6zdbq5
169228,1.198266e+18,ErvinLemark,Ervin Lemark,NaN,2019-11-23 15:45:01+00,0,0,0,The latest TopBitcoiner Daily! https://t.co/TI...,en,latest topbitcoin daily httpstcotiyn2wsdqw cry...


#### Check Samples

In [47]:
samples = dtf.sample(599)

In [48]:
print(remove_stopwords(samples["text\r"]))
print(list(samples["text_clean"]))

['@Dheeren29092403 @PMOIndia @narendramodi Are you interested in investing in Binary Options and bitcoin mining.\nInvest with a trusted account manager and earn huge profits weekly.\nInvest $500 earn $5000 weekly\nInvest\xa0$600\xa0earn $11,050\nYou can invest more.. OVER AND OVER AGAIN\nDM me if interested for more info.\r', 'Bitfury Releases Merchant, Developer Tools for Bitcoin’s Lighting Network https://www.altcoinflow.com/2019/01/23/bitfury-releases-merchant-developer-tools-for-bitcoins-lighting-network/\xa0…\r', 'NPXS/BTC is oversold on @binance\nIgnition sequence on\n#cryptocurrency #bitcoin #altcoins #btc pic.twitter.com/6AHZUrxMox\r', 'Crypto Markets Turn Green Following Tough Week, BTC Price Above $8,000 - https://t.co/OHuH39ioSS https://t.co/Lz8jC3vWkp\r', '******Tone Vays says TIME TO GET OUT OF BTC and SHORT BTC******BITCOIN IS GOING LOWER NOW******\nTrading Bitcoin - $11,000 Broken, Looking for More Downside https://t.co/XqrzxrLm0e via @YouTube\r', '@SelfKey @hitbtc This 

#### Export to CSV

In [49]:
dtf.to_csv('/content/drive/MyDrive/Colab Notebooks/Dataset/stopwords_stemming_lemmatisation_tweets_preprocessing_dataset.csv')

In [71]:
dtf

,id,user,fullname,url,timestamp,replies,likes,retweets,text\r,language,text_clean
1,1.127046e+18,mironyan,Yavruyan E,NaN,2019-05-11 01:00:33+00,0,0,0,!!!\r,en,
2,1.126824e+18,cryptobhoy,Kevin Gladstone,NaN,2019-05-10 10:19:35+00,0,0,0,@Beautyon_ Where's bitcoin sign guy when you n...,en,beautyon_ wh bitcoin sign guy need
3,1.132979e+18,MobileStarts,Mo.bi,NaN,2019-05-27 11:56:05+00,0,0,0,"New post: Top 3 Price Prediction Bitcoin, Ripp...",en,new post top 3 pric prediction bitcoin rippl e...
5,1.132980e+18,RhinoAK,RhinoAK,NaN,2019-05-27 12:00:36+00,0,0,0,Join me and start earning bitcoin by answering...,en,join start earning bitcoin answering paid emai...
7,1.132980e+18,DeribitRekt,DeribitRekt,NaN,2019-05-27 12:02:00+00,0,0,0,⏰ LIQUIDATION on BTC-PERPETUAL ☠️️\n\n Bought ...,en,liquidation btcperpet bought 10 btc 877800 27 ...
...,...,...,...,...,...,...,...,...,...,...,...
169223,1.198263e+18,btc_update,Bitcoin update,NaN,2019-11-23 15:32:37+00,0,0,0,Finland Prepares For Bitcoin Ransomware Attack...,en,finland prep bitcoin ransomw attack cyb fir dr...
169224,1.198264e+18,Cryptoevolucion,Cryptovolution,NaN,2019-11-23 15:36:23+00,0,0,0,#IoTex 🚨#UPDATE🚨 . Iotex is Very Stable and lo...,en,iotex updat iotex stabl look lik going uptrend...
169225,1.198265e+18,phuongdi2891,phuongdi2891,NaN,2019-11-23 15:39:21+00,0,0,0,Lydian Lion with the platform empowers human i...,en,lydian lion platform empow human interactiom r...
169226,1.198265e+18,emergeen,Emergeen DN,NaN,2019-11-23 15:40:50+00,0,0,0,Bitcoin wallet\n\n1KSrHvFavm1WB682ZjJNK355nJQe...,en,bitcoin wallet 1ksrhvfavm1wb682zjjnk355njqe6zdbq5
